# Autonomous Traders

An equity trading simulation, with 4 Traders and a Researcher, powered by a slew of MCP servers with tools & resources:

1. Our home-made Accounts MCP server (written by our engineering team!)
2. Fetch (get webpage via a local headless browser)
3. Memory
4. Brave Search
5. Financial data

And a resource to read information about the trader's account, and their investment strategy.

The goal of today's lab is to make a new python module, `traders.py` that will manage a single trader on our trading floor.

We will experiment and explore in the lab, and then migrate to a python module when we're ready.


In [11]:
import os
from dotenv import load_dotenv
from agents import Agent, Runner, trace, Tool
from agents.mcp import MCPServerStdio
from IPython.display import Markdown, display
from datetime import datetime
from accounts_client import read_accounts_resource, read_strategy_resource
from accounts import Account

load_dotenv(override=True)

True

### Let's start by gathering the MCP params for our trader

In [12]:
polygon_api_key = os.getenv("POLYGON_API_KEY")
polygon_plan = os.getenv("POLYGON_PLAN")

is_paid_polygon = polygon_plan == "paid"
is_realtime_polygon = polygon_plan == "realtime"

print(is_paid_polygon)
print(is_realtime_polygon)

False
False


In [13]:
if is_paid_polygon or is_realtime_polygon:
    market_mcp = {"command": "uvx","args": ["--from", "git+https://github.com/polygon-io/mcp_polygon@master", "mcp_polygon"], "env": {"POLYGON_API_KEY": polygon_api_key}}
else:
    market_mcp = ({"command": "uv", "args": ["run", "market_server.py"]})

trader_mcp_server_params = [
    {"command": "uv", "args": ["run", "accounts_server.py"]},
    {"command": "uv", "args": ["run", "push_server.py"]},
    market_mcp
]

### And now for our researcher

In [26]:
researcher_mcp_server_params = [
    {"command": "uvx", "args": ["mcp-server-fetch"]},
]

### Now create the MCPServerStdio for each

In [27]:
researcher_mcp_servers = [MCPServerStdio(params, client_session_timeout_seconds=30) for params in researcher_mcp_server_params]
trader_mcp_servers = [MCPServerStdio(params, client_session_timeout_seconds=30) for params in trader_mcp_server_params]
mcp_servers = trader_mcp_servers + researcher_mcp_servers

### Now let's make a Researcher Agent to do market research

And turn it into a tool - remember how this works for OpenAI Agents SDK, and the difference with handoffs?

In [28]:
async def get_researcher(mcp_servers) -> Agent:
    instructions = f"""You are a financial researcher. You are able to search the web for interesting financial news,
look for possible trading opportunities, and help with research.
Based on the request, you carry out necessary research and respond with your findings.
Take time to make multiple searches to get a comprehensive overview, and then summarize your findings.
If there isn't a specific request, then just respond with investment opportunities based on searching latest news.
The current datetime is {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}
"""
    researcher = Agent(
        name="Researcher",
        instructions=instructions,
        model="gpt-4.1-mini",
        mcp_servers=mcp_servers,
    )
    return researcher

In [29]:
async def get_researcher_tool(mcp_servers) -> Tool:
    researcher = await get_researcher(mcp_servers)
    return researcher.as_tool(
            tool_name="Researcher",
            tool_description="This tool researches online for news and opportunities, \
                either based on your specific request to look into a certain stock, \
                or generally for notable financial news and opportunities. \
                Describe what kind of research you're looking for."
        )

In [30]:
research_question = "What's the latest news on Apple?"

for server in researcher_mcp_servers:
    await server.connect()
researcher = await get_researcher(researcher_mcp_servers)
with trace("Researcher"):
    result = await Runner.run(researcher, research_question, max_turns=10)
display(Markdown(result.final_output))



Here's the latest summary on Apple Inc. (AAPL):

- Stock Closed (latest): $196.45, down 1.38% on high volume (46.7 million shares)
- Market Cap: Approximately $2.934 Trillion
- 52 Week Range: Low of $169.21 (April 2025) to High of $260.10 (December 2024)
- Dividend Yield: 0.53%, with dividend amount $0.26 per share
- Price to Earnings (P/E) Ratio (TTM): 27.75, Forward P/E: 26.92
- Earnings per share (EPS) last twelve months: $7.08
- Return on Equity (ROE): 152.55%
- Revenue (TTM): $400.366 Billion
- Net Margin: 26.86%
- Debt to Equity Ratio: 146.99%
- Next Earnings Date: Estimated July 30, 2025
- Stock showed a Year-to-Date percentage change of -21.55%, indicating it has faced some recent declines

There were no specific news headlines or major events fetched currently from the top financial news sources.

Would you like me to look specifically for recent news about Apple's product launches, earnings, legal issues, or other particular areas?

### Look at the trace

https://platform.openai.com/traces

In [37]:
merve_initial_strategy = "You are a day trader that aggressively buys and sells shares based on news and market conditions."
Account.get("Merve").reset(merve_initial_strategy)

display(Markdown(await read_accounts_resource("Merve")))
display(Markdown(await read_strategy_resource("Merve")))

{"name": "merve", "balance": 10000.0, "strategy": "You are a day trader that aggressively buys and sells shares based on news and market conditions.", "holdings": {}, "transactions": [], "portfolio_value_time_series": [["2025-06-15 13:00:45", 10000.0]], "total_portfolio_value": 10000.0, "total_profit_loss": 0.0}

You are a day trader that aggressively buys and sells shares based on news and market conditions.

### And now - to create our Trader Agent

In [38]:
agent_name = "Merve"

# Using MCP Servers to read resources
account_details = await read_accounts_resource(agent_name)
strategy = await read_strategy_resource(agent_name)

instructions = f"""
You are a trader that manages a portfolio of shares. Your name is {agent_name} and your account is under your name, {agent_name}.
You have access to tools that allow you to search the internet for company news, check stock prices, and buy and sell shares.
Your investment strategy for your portfolio is:
{strategy}
Your current holdings and balance is:
{account_details}
You have the tools to perform a websearch for relevant news and information.
You have tools to check stock prices.
You have tools to buy and sell shares.
You have tools to save memory of companies, research and thinking so far.
Please make use of these tools to manage your portfolio. Carry out trades as you see fit; do not wait for instructions or ask for confirmation.
"""

prompt = """
Use your tools to make decisions about your portfolio.
Investigate the news and the market, make your decision, make the trades, and respond with a summary of your actions.
"""

In [39]:
print(instructions)


You are a trader that manages a portfolio of shares. Your name is Merve and your account is under your name, Merve.
You have access to tools that allow you to search the internet for company news, check stock prices, and buy and sell shares.
Your investment strategy for your portfolio is:
You are a day trader that aggressively buys and sells shares based on news and market conditions.
Your current holdings and balance is:
{"name": "merve", "balance": 10000.0, "strategy": "You are a day trader that aggressively buys and sells shares based on news and market conditions.", "holdings": {}, "transactions": [], "portfolio_value_time_series": [["2025-06-15 13:00:45", 10000.0], ["2025-06-15 13:01:45", 10000.0]], "total_portfolio_value": 10000.0, "total_profit_loss": 0.0}
You have the tools to perform a websearch for relevant news and information.
You have tools to check stock prices.
You have tools to buy and sell shares.
You have tools to save memory of companies, research and thinking so fa

### And to run our Trader

In [40]:
for server in mcp_servers:
    await server.connect()

researcher_tool = await get_researcher_tool(researcher_mcp_servers)
trader = Agent(
    name=agent_name,
    instructions=instructions,
    tools=[researcher_tool],
    mcp_servers=trader_mcp_servers,
    model="gpt-4o-mini",
)
with trace(agent_name):
    result = await Runner.run(trader, prompt, max_turns=30)
display(Markdown(result.final_output))

### Summary of Actions

I conducted an analysis of current stock market conditions and identified potential day trading opportunities. Based on recent news and price momentum, I executed the following trades:

1. **Purchased Shares:**
   - **Intuit Inc (INTU)**
     - **Quantity:** 5 shares
     - **Price:** $755.49 (approximately)
     - **Rationale:** Recent positive earnings beat and elevated price targets from Wells Fargo; shares approaching all-time highs.

   - **Casella Waste Systems (CWST)**
     - **Quantity:** 15 shares
     - **Price:** $118.01 (approximately)
     - **Rationale:** Near all-time highs in a recession-resistant sector, anticipating increased investor attention with the Nasdaq opening bell coming up.

   - **Carpenter Technology Corporation (CRS)**
     - **Quantity:** 5 shares
     - **Price:** $249.22 (approximately)
     - **Rationale:** Strong earnings performance (beating estimates) over five consecutive quarters and analyst target upgrades suggest bullish movement.

### Portfolio Update
- **Current Holdings:**
  - **INTU:** 5 shares
  - **CWST:** 15 shares
  - **CRS:** 5 shares

- **Current Cash Balance:** $3206.39
- **Total Portfolio Value:** $9986.44
- **Total Profit/Loss:** -$13.56 (indicating a slight initial dip post-purchases)

I'll continue to monitor the news and market conditions to identify further trading opportunities. If you need updates or have other preferences, let me know!

### Then go and look at the trace

http://platform.openai.com/traces


In [41]:
# And let's look at the results of the trading

await read_accounts_resource(agent_name)

'{"name": "merve", "balance": 3206.3899, "strategy": "You are a day trader that aggressively buys and sells shares based on news and market conditions.", "holdings": {"INTU": 5, "CWST": 15, "CRS": 5}, "transactions": [{"symbol": "INTU", "quantity": 5, "price": 755.48796, "timestamp": "2025-06-15 13:03:42", "rationale": "Recent positive earnings beat, elevated price targets from Wells Fargo, and all-time highs offer a strong momentum for day trading."}, {"symbol": "CWST", "quantity": 15, "price": 118.00554, "timestamp": "2025-06-15 13:03:42", "rationale": "Approaching all-time highs in a recession-resistant sector. Anticipation of increased attention leading up to the Nasdaq opening bell."}, {"symbol": "CRS", "quantity": 5, "price": 249.21744, "timestamp": "2025-06-15 13:03:42", "rationale": "Strong earnings performance over five quarters and recent analysts\' target upgrades support bullish momentum."}], "portfolio_value_time_series": [["2025-06-15 13:00:45", 10000.0], ["2025-06-15 13:

### Now it's time to review the Python module made from this:

`mcp_params.py` is where the MCP servers are specified. You'll notice I've brought in some familiar friends: memory and push notifications!

`templates.py` is where the instructions and messages are set up (i.e. the System prompts and User prompts)

`traders.py` brings it all together.

You'll notice I've done something a bit fancy with code like this:

```
async with AsyncExitStack() as stack:
    mcp_servers = [await stack.enter_async_context(MCPServerStdio(params)) for params in mcp_server_params]
```

This is just a tidy way to combine our "with" statements (known as context managers) so that we don't need to do something ugly like this:

```
async with MCPServerStdio(params=params1) as mcp_server1:
    async with MCPServerStdio(params=params2) as mcp_server2:
        async with MCPServerStdio(params=params3) as mcp_server3:
            mcp_servers = [mcp_server1, mcp_server2, mcp_server3]
```

But it's equivalent.


In [2]:
from traders import Trader


In [3]:
trader = Trader("Ed")

In [ ]:
await trader.run()

In [ ]:
await read_accounts_resource("Ed")

### Now look at the trace

https://platform.openai.com/traces

### How many tools did we use in total?

In [ ]:
from mcp_params import trader_mcp_server_params, researcher_mcp_server_params

all_params = trader_mcp_server_params + researcher_mcp_server_params("ed")

count = 0
for each_params in all_params:
    async with MCPServerStdio(params=each_params, client_session_timeout_seconds=60) as server:
        mcp_tools = await server.list_tools()
        count += len(mcp_tools)
print(f"We have {len(all_params)} MCP servers, and {count} tools")